**FaceGate : Verifikasi Pengguna Menggunakan Citra Wajah**

FaceGate adalah aplikasi verifikasi wajah berbasis deep learning yang dikembangkan dengan Python. Aplikasi ini menggunakan model Keras untuk mengenali identitas wajah dari kamera secara real-time maupun dari gambar yang diunggah. Antarmukanya dibuat dengan Tkinter dan dilengkapi tampilan video serta tombol kontrol. Saat wajah dikenali dengan tingkat keyakinan di atas 90%, sistem menampilkan identitas dengan overlay visual; jika tidak, ditampilkan peringatan. FaceGate menggabungkan TensorFlow/Keras, OpenCV, dan Pillow, dan cocok digunakan sebagai prototipe sistem keamanan berbasis wajah.


**Import Library**

In [ ]:
import tkinter as tk
from tkinter import messagebox, filedialog
from PIL import Image, ImageTk, ImageDraw, ImageFont
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import os


**Memuat Model dan Label Kelas**

In [ ]:
model = load_model("keras_model.h5")

if os.path.exists("labels.txt"):
    with open("labels.txt", "r") as f:
        class_labels = [line.strip() for line in f.readlines()]
else:
    raise FileNotFoundError("File 'labels.txt' tidak ditemukan.")


Memuat model klasifikasi wajah dari file .h5, serta label kelas dari labels.txt. Label digunakan untuk menampilkan identitas yang dikenali oleh model.

**Membuat GUI Tkinter**

In [ ]:
window = tk.Tk()
window.title("FaceGate - Verifikasi Wajah")
window.geometry("800x620")
window.configure(bg="#1e1e1e")
window.resizable(False, False)

header = tk.Label(window, text="FaceGate", font=("Helvetica", 24, "bold"), fg="#00ff88", bg="#1e1e1e")
header.pack(pady=10)

subtext = tk.Label(window, text="Verifikasi Pengguna Menggunakan Citra Wajah", font=("Helvetica", 12), fg="white", bg="#1e1e1e")
subtext.pack(pady=5)

video_label = tk.Label(window, bg="black")
video_label.pack(pady=10)

button_frame = tk.Frame(window, bg="#1e1e1e")
button_frame.pack(pady=10)

start_button = tk.Button(button_frame, text="Mulai Verifikasi", width=20, bg="#00cc66", fg="white", font=("Helvetica", 10, "bold"))
stop_button = tk.Button(button_frame, text="Berhenti", width=20, bg="#cc3333", fg="white", font=("Helvetica", 10, "bold"))
upload_button = tk.Button(button_frame, text="Unggah Gambar", width=20, bg="#007acc", fg="white", font=("Helvetica", 10, "bold"))

start_button.grid(row=0, column=0, padx=10)
upload_button.grid(row=0, column=1, padx=10)


Membangun antarmuka pengguna: judul, subjudul, area tampilan video, dan tombol kontrol (mulai, berhenti, unggah gambar). Desain menggunakan warna gelap dan elemen modern.

**Fungsi Overlay Hasil Prediksi ke Gambar**

In [ ]:
def tampilkan_overlay(pil_img, status, nama=None):
    try:
        big_font = ImageFont.truetype("arialbd.ttf", 120)
        text_font = ImageFont.truetype("arial.ttf", 26)
    except:
        big_font = ImageFont.load_default()
        text_font = ImageFont.load_default()

    draw_img = pil_img.copy()
    if status == "dikenali":
        overlay = Image.new("RGBA", pil_img.size, (0, 102, 204, 130))
        draw_img = Image.alpha_composite(draw_img, overlay)
        draw = ImageDraw.Draw(draw_img)
        draw.text((draw_img.width // 2 - 40, 100), "✔", fill=(255, 255, 255, 255), font=big_font)
        draw.text((draw_img.width // 2 - 200, 300), f"Identitas Dikenali: {nama}", fill=(255, 255, 255, 255), font=text_font)
    else:
        overlay = Image.new("RGBA", pil_img.size, (0, 0, 0, 180))
        draw_img = Image.alpha_composite(draw_img, overlay)
        draw = ImageDraw.Draw(draw_img)
        draw.text((draw_img.width // 2 - 40, 100), "!", fill=(255, 0, 0, 255), font=big_font)
        draw.text((draw_img.width // 2 - 200, 300), "Identitas Tidak Dikenali", fill=(255, 255, 255, 255), font=text_font)

    return draw_img.convert("RGB")


Fungsi ini menambahkan lapisan teks ke gambar yang ditampilkan berdasarkan status prediksi: jika wajah dikenali (dikenali) atau tidak (tidak).

**Fungsi Update Frame dari Kamera**

In [ ]:
verifikasi_aktif = False
cap = None

def update_frame():
    if not verifikasi_aktif or cap is None:
        return

    ret, frame = cap.read()
    if not ret:
        return

    img = cv2.resize(frame, (224, 224)) / 255.0
    img = np.expand_dims(img, axis=0)

    pred = model.predict(img)
    class_idx = np.argmax(pred)
    confidence = float(pred[0][class_idx])

    display_frame = cv2.resize(frame, (720, 480))
    display_image = cv2.cvtColor(display_frame, cv2.COLOR_BGR2RGB)
    pil_img = Image.fromarray(display_image).convert("RGBA")

    if confidence > 0.9:
        name = class_labels[class_idx]
        pil_img = tampilkan_overlay(pil_img, "dikenali", name)
    else:
        pil_img = tampilkan_overlay(pil_img, "tidak")

    imgtk = ImageTk.PhotoImage(image=pil_img)
    video_label.imgtk = imgtk
    video_label.configure(image=imgtk)

    if verifikasi_aktif:
        window.after(30, update_frame)


Mengambil frame dari kamera, melakukan prediksi, menambahkan overlay hasil klasifikasi, dan menampilkannya ke GUI. Update terus setiap 30ms jika verifikasi aktif.

**Fungsi Tombol Kontrol Kamera dan Unggah Gambar**

In [ ]:
def mulai_verifikasi():
    global verifikasi_aktif, cap
    if cap is None or not cap.isOpened():
        cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        messagebox.showerror("Error", "Kamera tidak tersedia.")
        return

    verifikasi_aktif = True
    stop_button.grid(row=0, column=2, padx=10)
    update_frame()

def berhenti_verifikasi():
    global verifikasi_aktif, cap
    verifikasi_aktif = False
    if cap is not None and cap.isOpened():
        cap.release()
        cap = None
    stop_button.grid_forget()
    video_label.config(image='', bg="black")

def unggah_gambar():
    file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])
    if not file_path:
        return

    img_raw = cv2.imread(file_path)
    if img_raw is None:
        messagebox.showerror("Error", "Gagal memuat gambar.")
        return

    img_input = cv2.resize(img_raw, (224, 224)) / 255.0
    img_input = np.expand_dims(img_input, axis=0)

    pred = model.predict(img_input)
    class_idx = np.argmax(pred)
    confidence = float(pred[0][class_idx])

    display_frame = cv2.resize(img_raw, (720, 480))
    display_image = cv2.cvtColor(display_frame, cv2.COLOR_BGR2RGB)
    pil_img = Image.fromarray(display_image).convert("RGBA")

    if confidence > 0.9:
        name = class_labels[class_idx]
        pil_img = tampilkan_overlay(pil_img, "dikenali", name)
    else:
        pil_img = tampilkan_overlay(pil_img, "tidak")

    pil_img = pil_img.convert("RGB")
    imgtk = ImageTk.PhotoImage(image=pil_img)
    video_label.imgtk = imgtk
    video_label.configure(image=imgtk)


mulai_verifikasi() memulai kamera dan mulai klasifikasi.

berhenti_verifikasi() menghentikan kamera dan menghapus tampilan.

unggah_gambar() memungkinkan pengguna memilih gambar untuk diklasifikasikan tanpa kamera.

**Menghubungkan Fungsi ke Tombol**

In [ ]:
start_button.config(command=mulai_verifikasi)
stop_button.config(command=berhenti_verifikasi)
upload_button.config(command=unggah_gambar)


Menetapkan aksi tombol agar saat diklik, fungsi yang sesuai dijalankan (verifikasi dimulai, berhenti, atau unggah gambar).

**Menjalankan Antarmuka Aplikasi**

In [ ]:
window.mainloop()